# Open Street Map

By making use of the [OSMnx](https://osmnx.readthedocs.io/en/stable/) library we can fetch [OpenStreetMap](https://www.openstreetmap.org/) data and use it in [MGSurvE](https://github.com/Chipdelmal/MGSurvE)! This time, we will start by loading the library and setting up some of its internal parameters:

In [ ]:
# Fix PROJ path ---------------------------------------------------------------
import os; 
os.environ['PROJ_LIB']=r'/opt/conda/pkgs/proj-9.2.1-ha5fc9e9_0/share/proj'
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
# Load libraries --------------------------------------------------------------
import osmnx as ox
import numpy as np
import pandas as pd
from math import exp
from copy import deepcopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import MGSurvE as srv
ox.settings.log_console=False
ox.settings.use_cache=True
# Setup data path and ID ------------------------------------------------------
(ID, OUT_PTH) = ('Banfora', './out/')
srv.makeFolder(OUT_PTH)

## Downloading Buildings Footprints

In this example, we will download the data of a section of Banfora, Burkina Faso. To do this, we will center our reference point at the `(10.63524, -4.75195)` coordinates and set an altitude of `250` meters:

In [ ]:
(COORDS, DIST) = ((10.63524, -4.75195), 250)
# Scrape building footprints to dataframe -------------------------------------
BLD = ox.features.features_from_point(
    COORDS, tags={'building': True} , dist=DIST
)
NTW = ox.graph_from_point(
    COORDS, dist=DIST, network_type='all',
    retain_all=True, simplify=True, truncate_by_edge=True
)
# Extend the original dataframe to contain the centroids of geometries --------
BLD['centroid_lon'] = [poly.centroid.x for poly in BLD['geometry']]
BLD['centroid_lat'] = [poly.centroid.y for poly in BLD['geometry']]
BLD.reset_index(inplace=True)

The `BLD` dataframe contains all the information we need to generate a map of the location and we can, additionally, plot the roads by projecting the `NTW` graph:

In [ ]:
STYLE_BG = {'color': '#0b2545'}
STYLE_BD = {'color': '#faf9f9', 'alpha': 0.900}
STYLE_RD = {'color': '#ede0d4', 'alpha': 0.125, 'width': 1.5}
# Project and plot map --------------------------------------------------------
G = ox.project_graph(NTW, to_crs=ccrs.PlateCarree())
(fig, ax) = ox.plot_graph(
    G, node_size=0, figsize=(10, 100), show=False,
    bgcolor=STYLE_BG['color'], edge_color=STYLE_RD['color'], 
    edge_alpha=STYLE_RD['alpha'], edge_linewidth=STYLE_RD['width']
)
(fig, ax) = ox.plot_footprints(
    BLD, ax=ax, save=False, show=False, close=False,
    bgcolor=STYLE_BG['color'], color=STYLE_BD['color'], 
    alpha=STYLE_BD['alpha']
)

## Generating MGSurvE Landscape

If we wanted to make use of our data in [MGSurvE](https://github.com/Chipdelmal/MGSurvE), one simple way we can go about it is by using the `centroid_lon` and `centroid_lat` to generate our migration matrix and landscape:

In [ ]:
lonLats = np.array(list(zip(BLD['centroid_lon'], BLD['centroid_lat'])))
migDst = srv.calcDistanceMatrix(lonLats, distFun=srv.haversineDistance)

This time, we will define a custom exponential kernel. To do this we just need to define a function that takes the distance matrix as an argument, applies a function upon it based on the distance between every pair-wise combination of points (an exponential decay one, in this case), and returns a row-normalized migration probability matrix:

In [ ]:
# Defining our custom migration kernel ----------------------------------------
def exponentialKernel(distMat, decay):
    coordsNum = len(distMat)
    migrMat = np.empty((coordsNum, coordsNum))
    for (i, row) in enumerate(distMat):
        for (j, dst) in enumerate(row):
            migrMat[i][j] = exp(-decay*dst)
        for j in range(len(row)):
            if np.isnan(migrMat[i][j]):
                migrMat[i][j] = 0
    tauN = normalize(migrMat, axis=1, norm='l1')
    return np.array(tauN)
# Calculating the full migration matrix ---------------------------------------
MEAN_LIFETIME_DISPERSAL = 25
migMat = exponentialKernel(migDst, 1/MEAN_LIFETIME_DISPERSAL)

We now generate our sites dataframe:

In [ ]:
# Generate sites dataframe ----------------------------------------------------
sites = pd.DataFrame(lonLats, columns=['lon', 'lat'])
sites['t'] = [0]*sites.shape[0]
# Get the bounding box and centroid -------------------------------------------
BBOX = (
    (min(sites['lon']), max(sites['lon'])),
    (min(sites['lat']), max(sites['lat']))
)
CNTR = [i[0]+(i[1]-i[0])/2 for i in BBOX]

And define the traps we want to optimize:

In [ ]:
TRPS_NUM = 5
(initLon, initLat) = (
    sites.iloc[0]['lon']*TRPS_NUM, 
    sites.iloc[0]['lat']*TRPS_NUM
)
sid = [0]*TRPS_NUM
traps = pd.DataFrame({
    'sid': sid, 'lon': initLon, 'lat': initLat, 
    't': [0]*TRPS_NUM, 'f': [0]*TRPS_NUM
})
tKer = {0: {'kernel': srv.exponentialDecay, 'params': {'A': 0.5, 'b': 0.041674}}}

With these pieces in place, we can now define our landscape object:

In [ ]:
lnd = srv.Landscape(
    sites, 
    migrationMatrix=migMat, kernelFunction=exponentialKernel, 
    distanceFunction=srv.haversineDistance,
    kernelParams={'decay': 1/MEAN_LIFETIME_DISPERSAL},
    traps=traps, trapsKernels=tKer, trapsRadii=[0.250, 0.125, 0.100],
    landLimits=BBOX
)
bbox = lnd.getBoundingBox()
trpMsk = srv.genFixedTrapsMask(lnd.trapsFixed)

## Optimizing 

We will run discrete optimization upon our landscape as follows:

In [ ]:
GENS = 200
VERBOSE = True
POP_SIZE = int(10*(lnd.trapsNumber*1.5))
(MAT, MUT, SEL) = (
    {'cxpb':  0.300, 'indpb': 0.35}, 
    {'mutpb': 0.375, 'indpb': 0.50},
    {'tSize': 3}
)
lndGA = deepcopy(lnd)
# Running GA and exporting results --------------------------------------------
(lndDO, logbookDO) = srv.optimizeDiscreteTrapsGA(
    lndGA, pop_size=POP_SIZE, generations=GENS, verbose=VERBOSE,
    mating_params=MAT, mutation_params=MUT, selection_params=SEL,
    fitFuns={'inner': np.sum, 'outer': np.mean}
)
srv.exportLog(logbookDO, OUT_PTH, '{}_LOG_DO'.format(ID))
srv.dumpLandscape(lndDO, OUT_PTH, '{}_LND_DO'.format(ID), fExt='pkl')

And plot our results!

In [ ]:
(fig, ax) = (
    plt.figure(figsize=(10, 10), facecolor=STYLE_BG['color']), 
    plt.axes(projection=ccrs.PlateCarree())
)
G = ox.project_graph(NTW, to_crs=ccrs.PlateCarree())
(fig, ax) = ox.plot_graph(
    G, ax, node_size=0, figsize=(40, 40), show=False,
    bgcolor=STYLE_BG['color'], edge_color=STYLE_RD['color'], 
    edge_alpha=STYLE_RD['alpha'], edge_linewidth=STYLE_RD['width']
)
(fig, ax) = ox.plot_footprints(
    BLD, ax=ax, save=False, show=False, close=False,
    bgcolor=STYLE_BG['color'], color=STYLE_BD['color'], 
    alpha=STYLE_BD['alpha']
)
lndDO.plotTraps(
    fig, ax, 
    size=500, zorders=(30, 25), transparencyHex='99', 
    proj=ccrs.PlateCarree()
)
srv.plotClean(fig, ax, bbox=BBOX)
ax.set_facecolor(STYLE_BG['color'])
fig.savefig(
    os.path.join(OUT_PTH, '{}DO.png'.format(ID)), 
    facecolor='w', bbox_inches='tight', pad_inches=0.1, dpi=150
)

Running the continuous case is also easy!

In [ ]:
# Optimize the continuous landscape -------------------------------------------
(lndCO, logbookCO) = srv.optimizeTrapsGA(
    lndGA, 
    pop_size='auto', generations=GENS,
    mating_params='auto', mutation_params='auto', selection_params='auto',
    fitFuns={'inner': np.sum, 'outer': np.mean}, verbose=VERBOSE
)
srv.exportLog(logbookCO, OUT_PTH, '{}_LOG_CO'.format(ID))
srv.dumpLandscape(lndCO, OUT_PTH, '{}_LND_CO'.format(ID), fExt='pkl')
# Plot the results ------------------------------------------------------------
(fig, ax) = (
    plt.figure(figsize=(10, 10), facecolor=STYLE_BG['color']), 
    plt.axes(projection=ccrs.PlateCarree())
)
G = ox.project_graph(NTW, to_crs=ccrs.PlateCarree())
(fig, ax) = ox.plot_graph(
    G, ax, node_size=0, figsize=(40, 40), show=False,
    bgcolor=STYLE_BG['color'], edge_color=STYLE_RD['color'], 
    edge_alpha=STYLE_RD['alpha'], edge_linewidth=STYLE_RD['width']
)
(fig, ax) = ox.plot_footprints(
    BLD, ax=ax, save=False, show=False, close=False,
    bgcolor=STYLE_BG['color'], color=STYLE_BD['color'], 
    alpha=STYLE_BD['alpha']
)
lndDO.plotTraps(
    fig, ax, 
    size=500, zorders=(30, 25), transparencyHex='99', 
    proj=ccrs.PlateCarree()
)
srv.plotClean(fig, ax, bbox=BBOX)
ax.set_facecolor(STYLE_BG['color'])
fig.savefig(
    os.path.join(OUT_PTH, '{}CO.png'.format(ID)), 
    facecolor='w', bbox_inches='tight', pad_inches=0.1, dpi=150
)

<hr>

# More Information

* [OSMnx Documentation](https://osmnx.readthedocs.io/)
* [OpenStreetMap](https://www.openstreetmap.org)
